In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, LeakyReLU
from keras.layers.noise import AlphaDropout
from keras.utils.generic_utils import get_custom_objects
from keras import backend as K
from keras.optimizers import Adam, SGD

def preprocess_mnist(x_train, y_train, x_test, y_test):
    x_train = x_train.reshape(x_train.shape[0], 28 * 28)
    x_test = x_test.reshape(x_test.shape[0], 28 * 28)
    input_shape = (28 * 28,)
    
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    
    x_train /= 255
    x_test /= 255
    
    y_train = to_categorical(y_train)
    y_test= to_categorical(y_test)
    
    return x_train, y_train, x_test, y_test, input_shape

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train, x_test, y_test, input_shape = preprocess_mnist(x_train, y_train, x_test, y_test)

def build_cnn(activation,
              dropout_rate,
              optimizer):
    model = Sequential()
    
    if(activation == 'selu'):
        model.add(Dense(512, activation=activation, input_shape=input_shape, kernel_initializer='lecun_normal'))
        model.add(AlphaDropout(0.25))
        model.add(Dense(256, activation=activation, kernel_initializer='lecun_normal'))
        model.add(AlphaDropout(0.25))
        model.add(Dense(128, activation=activation, kernel_initializer='lecun_normal'))
        model.add(AlphaDropout(0.5))
        model.add(Dense(10, activation='softmax'))
    else:
        model.add(Dense(512, activation=activation, input_shape=input_shape, kernel_initializer='lecun_normal'))
        model.add(Dropout(0.25))
        model.add(Dense(256, activation=activation, kernel_initializer='lecun_normal'))
        model.add(Dropout(0.25))
        model.add(Dense(128, activation=activation, kernel_initializer='lecun_normal'))
        model.add(Dropout(0.5))
        model.add(Dense(10, activation='softmax'))
    
    model.compile(
        loss='categorical_crossentropy', 
        optimizer=optimizer, 
        metrics=['accuracy']
    )
    
    return model


def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

act_func = ['sigmoid', 'tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

result = []


for activation in act_func:
    print('\nTraining with -->{0}<-- activation function\n'.format(activation))
    
    model = build_cnn(activation=activation,
                      dropout_rate=0.2,
                      optimizer=SGD())
    
    history = model.fit(x_train, y_train,
          validation_split=0.20,
          batch_size=128,
          epochs=50,
          verbose=1,
          validation_data=(x_test, y_test))
    
    result.append(history)
    
    K.clear_session()
    del model

for r in result:
    print(r.history)

11501568/11490434 [==============================] - 0s 0us/step

Training with -->sigmoid<-- activation function

Epoch 1/50
375/375 [==============================] - 16s 5ms/step - loss: 2.5402 - accuracy: 0.1008 - val_loss: 2.3000 - val_accuracy: 0.1060
Epoch 2/50
375/375 [==============================] - 1s 4ms/step - loss: 2.4102 - accuracy: 0.1046 - val_loss: 2.2986 - val_accuracy: 0.1060
Epoch 3/50
375/375 [==============================] - 1s 3ms/step - loss: 2.3694 - accuracy: 0.1071 - val_loss: 2.2972 - val_accuracy: 0.1060
Epoch 4/50
375/375 [==============================] - 1s 3ms/step - loss: 2.3499 - accuracy: 0.1075 - val_loss: 2.2961 - val_accuracy: 0.1060
Epoch 5/50
375/375 [==============================] - 1s 3ms/step - loss: 2.3360 - accuracy: 0.1035 - val_loss: 2.2954 - val_accuracy: 0.1060
Epoch 6/50
375/375 [==============================] - 1s 3ms/step - loss: 2.3276 - accuracy: 0.1071 - val_loss: 2.2949 - val_accuracy: 0.1060
Epoch 7/50
375/375 [============